In [1]:
from canopus.utils import tket_to_qiskit, qiskit_to_tket
from canopus.synthesis import *
from pytket.utils import compare_unitaries

def pytket_qft_circuit(n):
    from qiskit.circuit.library import QFT
    qc = QFT(n, do_swaps=False).decompose()
    return qiskit_to_tket(qc)

from pytket import Circuit
from pytket.extensions.qiskit import AerBackend
from pytket.passes import PlacementPass, DefaultMappingPass, RoutingPass
from pytket.predicates import ConnectivityPredicate
from pytket.placement import GraphPlacement
from pytket import architecture


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [ ]:
circ = pytket_qft_circuit(6)
arch = architecture.SquareGrid(6, 3)

In [ ]:
RoutingPass(arch).apply(circ)

pytket._tket.architecture.FullyConnected

In [ ]:
from pytket import architecture

In [14]:
architecture.FullyConnected(10).nodes

[fcNode[0],
 fcNode[1],
 fcNode[2],
 fcNode[3],
 fcNode[4],
 fcNode[5],
 fcNode[6],
 fcNode[7],
 fcNode[8],
 fcNode[9]]

In [2]:
from qiskit.transpiler.basepasses import TransformationPass  
from qiskit.dagcircuit import DAGCircuit  
from qiskit.circuit import QuantumCircuit, QuantumRegister  
from qiskit.circuit.library import UnitaryGate  
from qiskit.converters import circuit_to_dag  
import numpy as np  
  
class CustomTwoQubitDecompose(TransformationPass):  
    """自定义两比特门分解pass，使用用户提供的4x4 unitary matrix分解函数"""  
      
    def __init__(self, custom_decompose_func):  
        """  
        Args:  
            custom_decompose_func: 用户自定义的4x4矩阵分解函数  
                                 输入: 4x4 numpy array  
                                 输出: QuantumCircuit对象  
        """  
        super().__init__()  
        self.custom_decompose_func = custom_decompose_func  
      
    def run(self, dag: DAGCircuit) -> DAGCircuit:  
        """运行分解pass"""  
        # 遍历DAG中的所有操作节点  
        for node in dag.op_nodes():  
            # 检查是否为两比特UnitaryGate  
            if (isinstance(node.op, UnitaryGate) and   
                len(node.qargs) == 2):  
                  
                # 获取unitary矩阵  
                unitary_matrix = node.op.to_matrix()  
                  
                # 使用自定义分解函数  
                decomposed_circuit = self.custom_decompose_func(unitary_matrix)  
                  
                # 转换为DAG并替换原节点  
                decomposed_dag = circuit_to_dag(decomposed_circuit)  
                dag.substitute_node_with_dag(node, decomposed_dag)  
          
        return dag  
  
# 示例：用户的自定义分解函数  
def my_4x4_decompose_function(unitary_4x4):  
    """  
    用户自定义的4x4 unitary矩阵分解函数  
    这里只是一个示例实现，您需要替换为您的实际分解算法  
    """  
    # 创建2比特量子电路  
    qc = QuantumCircuit(2)  
      
    # 这里是您的分解逻辑的占位符  
    # 实际实现中，您需要将4x4矩阵分解为基本门的序列  
    # 例如：使用KAK分解、Weyl分解等  
      
    # 示例：简单的分解（实际使用时请替换为您的算法）  
    # 这里只是演示如何构建返回的电路  
    qc.h(0)  
    qc.cx(0, 1)  
    qc.rz(0.1, 0)  # 参数应该从您的分解算法中计算得出  
    qc.ry(0.2, 1)  # 参数应该从您的分解算法中计算得出  
      
    return qc  
  
# 使用示例  
if __name__ == "__main__":  
    from qiskit import QuantumCircuit  
    from qiskit.converters import circuit_to_dag, dag_to_circuit  
      
    # 创建包含两比特UnitaryGate的测试电路  
    test_circuit = QuantumCircuit(2)  
      
    # 添加一个随机的两比特unitary门  
    random_unitary = np.random.rand(4, 4) + 1j * np.random.rand(4, 4)  
    # 使其成为真正的unitary矩阵（这里简化处理）  
    u, s, vh = np.linalg.svd(random_unitary)  
    unitary_matrix = u @ vh  
      
    test_circuit.append(UnitaryGate(unitary_matrix), [0, 1])  
      
    # 创建自定义分解pass  
    custom_pass = CustomTwoQubitDecompose(my_4x4_decompose_function)  
      
    # 应用pass  
    dag = circuit_to_dag(test_circuit)  
    decomposed_dag = custom_pass.run(dag)  
    decomposed_circuit = dag_to_circuit(decomposed_dag)  
      
    print("原始电路:")  
    print(test_circuit)  
    print("\n分解后电路:")  
    print(decomposed_circuit)

原始电路:
     ┌──────────┐
q_0: ┤0         ├
     │  Unitary │
q_1: ┤1         ├
     └──────────┘

分解后电路:
     ┌───┐     ┌─────────┐
q_0: ┤ H ├──■──┤ Rz(0.1) ├
     └───┘┌─┴─┐├─────────┤
q_1: ─────┤ X ├┤ Ry(0.2) ├
          └───┘└─────────┘
